In [1]:
import requests
import urllib3
import csv
import time
from bs4 import BeautifulSoup
from datetime import date, timedelta
from tempfile import NamedTemporaryFile
import shutil
t = date.today()
today = t.strftime("%d%m%y")
urllib3.disable_warnings()
url = "https://kworb.net/itunes/artist/taylorswift.html"
htmlContent = requests.get(url)
soup=BeautifulSoup(htmlContent.content, 'html.parser')

In [17]:
soup.body.find_all('td')[4:][0].find_all("div", class_="itu")[1:][7].a.text.split(' ',1)

['#2', 'United States']

In [18]:
rapline=["Intro : Persona", "Jamais vu", "Interlude : Shadow", "UGH", "Respect", "Outro : Ego", "Dionysus",
         "Trivia 起 : Just Dance", "Trivia 轉 : Seesaw","Trivia 承 : Love","Her","Tear","Intro: Boy Meets Evil",
         "First Love", "Reflection", "MAMA","BTS Cypher 4","Intro: Never Mind",
         "Intro: The Most Beautiful Moment in Life","Intro: What Am I to You","BTS Cypher, Pt. 3: Killer",
         "Intro: Skool Luv Affair","BTS Cypher, Pt.2: Triptych","Intro: O!RUL8,2?","Paldogangsan",
         "BTS Cypher, Pt.1","All Night",
         "A Brand New Day"]

In [24]:
def currchart(soup,chart):
    currentcharting={}
    tableinfo=soup.body.find_all('td')
    for release in tableinfo[4:]:
        if release.div==None:
            continue
        else:
            title=release.div.text
            itunespositions=release.find_all("div", class_=chart)[1:]
            currentcharting[title]={}
            for position in itunespositions:
                working=position.a.text.split(" ",1)
                working[0]=int(working[0].lstrip("#"))
                country=working[1]
                currentcharting[title][country]={}
                currentcharting[title][country]["pos"]=working[0]
                currentcharting[title][country]["mov"]=position.span.text
                #currentcharting[title][country].append(working[0])
                #currentcharting[title][country].append(position.span.text)
    #for title in currentcharting:
        #if title in rapline:
            #print(title)
            #for country in currentcharting[title]:
                #pos=str(currentcharting[title][country]["pos"])
               # mov=currentcharting[title][country]["mov"]
                #print(country +" "+ pos +' ' + mov)
            #print("\n")
        #else:
           # continue
    return currentcharting

In [25]:
def record(currentcharting):
    for title in currentcharting:
        thecsv = f"/Users/ajkoenig/iTunes/{title}.csv"
        try:
            tempfile=NamedTemporaryFile(mode='w',delete=False)
            fields=['country', 'peak', 'date']
            with open(thecsv, mode='r') as infile, tempfile:
                reader = csv.DictReader(infile)
                writer = csv.DictWriter(tempfile, fieldnames=fields)
                writer.writeheader()
                historiccountries=[]
                for row in reader:
                    historiccountries.append(row['country'])
                    #in historic & in current check (skips if not currently charting)
                    if row['country'] in currentcharting[title].keys():
                        #current higher than historic check
                        if int(row['peak']) > currentcharting[title][row['country']]['pos']:
                            print("New peak: "+title+" "+row['country']+ " "+str(currentcharting[title][row['country']]['pos']))
                            row['country'], row['peak'], row['date'] = row['country'], currentcharting[title][row['country']]['pos'], today
                    row = {'country': row['country'], 'peak': row['peak'], 'date': row['date']}
                    writer.writerow(row)
                #in current not historic check
                for country in list(currentcharting[title]):
                    if country not in historiccountries:
                        print("New entry: "+title+" "+country+ " "+str(currentcharting[title][country]['pos']))
                        row = {'country': country, 'peak':currentcharting[title][country]['pos'], 'date':today}
                        writer.writerow(row)
            shutil.move(tempfile.name, thecsv) 
        except:
            print("New song:"+title)
            with open(thecsv, 'w') as csvfile:
                fieldnames = ['country', 'peak','date']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for country in currentcharting[title]:
                    writer.writerow({'country': country, 'peak':currentcharting[title][country]['pos'], 'date':today})
    return

In [22]:
currentcharting=currchart(soup)
distest=record(currentcharting)
distest

NameError: name 'record' is not defined

In [ ]:
while True:
    t = date.today()
    today = t.strftime("%d%m%y")
    htmlContent = requests.get(url)
    soup=BeautifulSoup(htmlContent.content, 'html.parser')
    currentcharting=currchart(soup)
    record(currentcharting)
    print('rest')
    time.sleep(300)

rest
rest
rest
rest
rest
rest
New peak: Friends Qatar 4
New peak: FAKE LOVE Bolivia 7
New peak: FAKE LOVE Cambodia 161
New peak: Dionysus United Arab Emirates 2
rest
rest
rest
New peak: Album: MAP OF THE SOUL : 7 ~ THE JOURNEY ~ Moldova 2
New peak: Album: MAP OF THE SOUL : PERSONA Armenia 130
New peak: Album: Love Yourself 轉 'Tear' Armenia 136
rest
rest
rest
New peak: Respect Bulgaria 1
New peak: Black Swan Kazakhstan 88
rest
rest
rest
New entry: Outro : Ego Russia 38
rest
rest
rest
New peak: ON Bahrain 5
New song:Skit: Circle Room Talk
New peak: Album: MAP OF THE SOUL : 7 Singapore 157
New peak: Album: Love Yourself 結 'Answer' Singapore 152
New peak: Album: MAP OF THE SOUL : PERSONA Fiji 99
New peak: Album: Love Yourself 轉 'Tear' Singapore 151
rest
rest
rest
rest
rest
rest
New entry: Magic Shop Singapore 1
New peak: Black Swan Kazakhstan 84
New song:Singularity
New peak: Heartbeat Laos 62
rest
rest
rest
New entry: Filter Turkey 131
rest
rest
rest
New peak: Spring Day Malaysia 62
New p

In [26]:
currentcharting=currchart(soup,'itu')
title='cardigan'
thecsv = f"/Users/ajkoenig/iTunes/{title}.csv"
try:
    tempfile=NamedTemporaryFile(mode='w',delete=False)
    fields=['country', 'peak', 'date']
    with open(thecsv, mode='r') as infile, tempfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(tempfile, fieldnames=fields)
        writer.writeheader()
        historiccountries=[]
        for row in reader:
            historiccountries.append(row['country'])
            #in historic & in current check (skips if not currently charting)
            if row['country'] in currentcharting[title].keys():
                #current higher than historic check
                if int(row['peak']) > currentcharting[title][row['country']]['pos']:
                    print("New peak: "+title+" "+row['country']+ " "+str(currentcharting[title][row['country']]['pos']))
                    row['country'], row['peak'], row['date'] = row['country'], currentcharting[title][row['country']]['pos'], today
            row = {'country': row['country'], 'peak': row['peak'], 'date': row['date']}
            writer.writerow(row)
        #in current not historic check
        for country in list(currentcharting[title]):
            if country not in historiccountries:
                print("New entry: "+title+" "+country+ " "+str(currentcharting[title][country]['pos']))
                row = {'country': country, 'peak':currentcharting[title][country]['pos'], 'date':today}
                writer.writerow(row)
    shutil.move(tempfile.name, thecsv) 
except:
    print("New song:"+title)
    with open(thecsv, 'w') as csvfile:
        fieldnames = ['country', 'peak','date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for country in currentcharting[title]:
            writer.writerow({'country': country, 'peak':currentcharting[title][country]['pos'], 'date':today})

New entry: cardigan Cambodia 28
New entry: cardigan Costa Rica 1
New entry: cardigan Israel 1
New entry: cardigan Malaysia 7
New entry: cardigan Mexico 46
New entry: cardigan Saudi Arabia 2
New entry: cardigan Vietnam 6
New entry: cardigan United States 79
New entry: cardigan Hong Kong 31
New entry: cardigan Thailand 19
New entry: cardigan Bolivia 4
New entry: cardigan Colombia 4
New entry: cardigan Canada 79
New entry: cardigan Netherlands 177
New entry: cardigan Norway 10
New entry: cardigan Finland 25
New entry: cardigan Germany 30
New entry: cardigan Egypt 31
New entry: cardigan Belgium 34
New entry: cardigan South Africa 38
New entry: cardigan Taiwan 41
New entry: cardigan El Salvador 113
New entry: cardigan Russia 234
New entry: cardigan Italy 290


In [ ]:
currentcharting2={'goodsong':{'USA':{'pos': 1, 'mov':'(NE)'}, 'UK':{'pos':2, 'mov':'(NE)'}, 'France':{'pos':4, 'mov':'(-1)'}}}

In [ ]:
thecsv = f"/Users/ajkoenig/iTunes/goodsong.csv"
tempfile=NamedTemporaryFile(mode='w',delete=False)
fields=['country', 'peak', 'date']
with open(thecsv, mode='r') as infile, tempfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(tempfile, fieldnames=fields)
    writer.writeheader()
    historiccountries=[]
    for row in reader:
        historiccountries.append(row['country'])
        #in historic & in current check (skips if not currently charting)
        if row['country'] in currentcharting2['goodsong'].keys():
            #current higher than historic check
            if int(row['peak']) > currentcharting2['goodsong'][row['country']]['pos']:
                print("New peak:"+" goodsong"+" "+row['country']+ " "+str(currentcharting2['goodsong'][row['country']]['pos']))
                row['country'], row['peak'], row['date'] = row['country'], currentcharting2['goodsong'][row['country']]['pos'], today
        row = {'country': row['country'], 'peak': row['peak'], 'date': row['date']}
        writer.writerow(row)
    #in current not historic check
    for country in list(currentcharting2['goodsong']):
        if country not in historiccountries:
            print("New entry:"+' goodsong'+" "+country+ " "+str(currentcharting2['goodsong'][country]['pos']))
            row = {'country': country, 'peak':currentcharting2['goodsong'][country]['pos'], 'date':today}
            writer.writerow(row)
shutil.move(tempfile.name, thecsv)

In [ ]:
historiccountries

In [ ]:
list(currentcharting2['goodsong'].keys())